In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import PyUber
from datetime import datetime
import glob
import os
import keras
from keras.layers import Activation, Dense, Conv1D, LSTM, Dropout, Flatten

Using TensorFlow backend.


In [2]:
model = keras.models.Sequential()
model.add(Dense(32, input_dim=20,activation='sigmoid'))
#model.add(LSTM(36, return_sequences=True))
#model.add(LSTM(16))
#model.add(LSTM(8))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam',loss='mse')

In [3]:
model

In [4]:
model.layers

In [1]:

stats.zscore([1,2])

array([-1.,  1.])

In [3]:
sta = pd.read_csv('training_data/cleaned_charts_training.csv')

In [4]:
sta.head()

,key,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,trending,burst,elevated,clean
0,DE-PAT-FTA.TA.LAT426_PM3.2020-03-12,0.886886,0.779347,0.161230,0.483691,0.161230,0.000000,0.483691,0.000000,0.483691,...,0.483691,0.628798,0.000000,0.271539,0.000000,0.000000,0.0,0.0,0.0,1.0
1,DE-LAT-TNC.TA.PAT408_PM5.2020-04-11,0.179424,0.159488,0.119616,0.179424,0.239231,0.478463,0.119616,0.059808,0.119616,...,0.102528,0.166702,0.166652,0.167073,0.119616,0.239231,0.0,0.0,0.0,1.0
2,DE-LAT.TA.LAT416_PM1.2020-04-11,1.178032,0.642563,1.285126,0.749657,1.136274,1.966632,0.880549,1.308925,0.428375,...,2.243871,0.939131,0.856751,0.428375,1.465495,0.821053,0.0,0.0,0.0,1.0
3,DE-PAT-BE.TA.PAT430_PM4.2020-04-11,0.000000,0.243276,0.241994,0.000000,0.245945,0.540329,0.000000,0.268161,0.000000,...,1.891150,0.540329,0.000000,0.771898,0.000000,0.540329,1.0,0.0,0.0,0.0
4,DE-PAT-BE.TA.PAT430_PM5.2020-04-11,0.446321,0.500431,0.000000,0.584223,0.000000,1.729052,0.294725,0.654091,0.651201,...,0.540329,0.402964,0.000000,0.258753,0.000000,0.000000,0.0,0.0,0.0,1.0


In [7]:
sta.is_dirty.sum()

192

In [5]:
y = sta.trending.values

In [6]:
x = sta.drop(['key', 'trending', 'burst', 'elevated', 'clean'], axis=1).values

In [7]:
x.shape

(10310, 20)

In [11]:
kfit = model.fit(x, y, epochs=50, shuffle=True, verbose=1)

Epoch 1/50
1647/1647 [==============================] - 0s 244us/step - loss: 0.1842
Epoch 2/50
1647/1647 [==============================] - 0s 71us/step - loss: 0.1045
Epoch 3/50
1647/1647 [==============================] - 0s 80us/step - loss: 0.1023
Epoch 4/50
1647/1647 [==============================] - 0s 81us/step - loss: 0.0990
Epoch 5/50
1647/1647 [==============================] - 0s 75us/step - loss: 0.0956
Epoch 6/50
1647/1647 [==============================] - 0s 68us/step - loss: 0.0917
Epoch 7/50
1647/1647 [==============================] - 0s 74us/step - loss: 0.0870
Epoch 8/50
1647/1647 [==============================] - 0s 71us/step - loss: 0.0821
Epoch 9/50
1647/1647 [==============================] - 0s 84us/step - loss: 0.0771
Epoch 10/50
1647/1647 [==============================] - 0s 77us/step - loss: 0.0729
Epoch 11/50
1647/1647 [==============================] - 0s 74us/step - loss: 0.0689
Epoch 12/50
1647/1647 [==============================] - 0s 74us/step - l

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                672       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.save('ss_trend_4_Dense.h5')

In [8]:
x1=x.reshape(-1,20,1)

In [9]:
y.shape

(10310,)

In [10]:
y.reshape(-1,1).shape

(10310, 1)

In [11]:
x1.shape

(10310, 20, 1)

In [12]:
x1.T.shape

(1, 20, 10310)

### Shape that worked: 

y.shape = (1647,)

x1.shape = (1647,20,1)

In [19]:
model = keras.models.Sequential()
#model.add(Dense(20, input_dim=20,activation='sigmoid'))
model.add(LSTM(8, return_sequences=True, activation='tanh', input_shape=(20,1))) #, input_shape=(20)
model.add(Dropout(0.2))
model.add(LSTM(4, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(4, return_sequences=True, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(LSTM(4, return_sequences=False))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam',loss='mse')

In [22]:
kfit = model.fit(x1, y, epochs=200, shuffle=True, verbose=1)

Epoch 1/200
10310/10310 [==============================] - 6s 589us/step - loss: 0.0850
Epoch 2/200
10310/10310 [==============================] - 6s 601us/step - loss: 0.0858
Epoch 3/200
10310/10310 [==============================] - 7s 691us/step - loss: 0.0853
Epoch 4/200
10310/10310 [==============================] - 7s 712us/step - loss: 0.0843
Epoch 5/200
10310/10310 [==============================] - 7s 725us/step - loss: 0.0850
Epoch 6/200
10310/10310 [==============================] - 7s 721us/step - loss: 0.0836
Epoch 7/200
10310/10310 [==============================] - 8s 728us/step - loss: 0.0849
Epoch 8/200
10310/10310 [==============================] - 8s 787us/step - loss: 0.0848
Epoch 9/200
10310/10310 [==============================] - 8s 769us/step - loss: 0.0841
Epoch 10/200
10310/10310 [==============================] - 7s 703us/step - loss: 0.0860
Epoch 11/200
10310/10310 [==============================] - 7s 701us/step - loss: 0.0835
Epoch 12/200
10310/10310 [====

10310/10310 [==============================] - 10s 930us/step - loss: 0.0783
Epoch 183/200
10310/10310 [==============================] - 10s 958us/step - loss: 0.0787
Epoch 184/200
10310/10310 [==============================] - 10s 975us/step - loss: 0.0778
Epoch 185/200
10310/10310 [==============================] - 10s 984us/step - loss: 0.0784
Epoch 186/200
10310/10310 [==============================] - 10s 939us/step - loss: 0.0780
Epoch 187/200
10310/10310 [==============================] - 10s 929us/step - loss: 0.0786
Epoch 188/200
10310/10310 [==============================] - 10s 961us/step - loss: 0.0770
Epoch 189/200
10310/10310 [==============================] - 10s 960us/step - loss: 0.0782
Epoch 190/200
10310/10310 [==============================] - 10s 953us/step - loss: 0.0780
Epoch 191/200
10310/10310 [==============================] - 10s 981us/step - loss: 0.0784
Epoch 192/200
10310/10310 [==============================] - 10s 986us/step - loss: 0.0786
Epoch 193/200

In [23]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 20, 8)             320       
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 8)             0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 4)             208       
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 4)             0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 20, 4)             144       
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 4)             0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 4)                

In [24]:
model.save('ss_trend_5LYRLSTM_20210426.h5')

(1647, 20, 1)